# Lab 06-1 Softmax Classification Eager
* Softmax를 사용하여분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [4]:
#Weight and bias setting
W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [5]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385160e-01 1.67723209e-01 4.51891690e-01]
 [3.23390663e-01 5.90759777e-02 6.17533386e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520462e-02 1.07279727e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94076904e-06 3.81133276e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [7]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [8]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [9]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [10]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=118, shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914616, -0.6509784 ,  0.58183223],
       [-1.5221257 , -1.214863  ,  2.7369888 ],
       [-1.2473826 , -1.7611003 ,  3.008483  ],
       [-1.2014605 , -1.8659232 ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: id=116, shape=(3,), dtype=float32, numpy=array([-0.15212911, -0.342192  ,  0.49432108], dtype=float32)>]


In [19]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 0.126001
Loss at epoch 100: 0.124082
Loss at epoch 200: 0.122203
Loss at epoch 300: 0.120380
Loss at epoch 400: 0.118610
Loss at epoch 500: 0.116891
Loss at epoch 600: 0.115221
Loss at epoch 700: 0.113598
Loss at epoch 800: 0.112019
Loss at epoch 900: 0.110482
Loss at epoch 1000: 0.108987
Loss at epoch 1100: 0.107531
Loss at epoch 1200: 0.106112
Loss at epoch 1300: 0.104730
Loss at epoch 1400: 0.103382
Loss at epoch 1500: 0.102067
Loss at epoch 1600: 0.100785
Loss at epoch 1700: 0.099534
Loss at epoch 1800: 0.098313
Loss at epoch 1900: 0.097120
Loss at epoch 2000: 0.095956


# Prediction Check

In [20]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor(
[[0.00783868 0.08934675 0.90281457]
 [0.00837632 0.10334261 0.88828105]
 [0.00890698 0.11949147 0.8716016 ]
 [0.01300187 0.34645918 0.64053893]
 [0.01916472 0.3473155  0.6335198 ]
 [0.01627325 0.38224185 0.6014849 ]
 [0.03545809 0.24856271 0.7159792 ]
 [0.04392094 0.21500538 0.74107367]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 2 2 2 2 2], shape=(8,), dtype=int64)


In [21]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[1.17311458e-06 4.64814686e-04 9.99534011e-01]
 [8.37631337e-03 1.03342585e-01 8.88281107e-01]
 [1.04251918e-09 1.33687243e-01 8.66312802e-01]
 [5.01113000e-08 8.58640432e-01 1.41359538e-01]
 [1.16531663e-01 8.71398807e-01 1.20694786e-02]
 [5.55007122e-02 9.44499254e-01 7.49097243e-11]
 [8.76283109e-01 1.23716690e-01 2.13872141e-07]
 [9.74358737e-01 2.56412514e-02 1.90751481e-09]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


# Convert as Class

In [14]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            
            return grads
    
def fit(self, X, Y, epochs=2000, verbose=500):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = self.grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, self.variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, self.loss_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)